In [6]:
library(tidyverse)
library(caret)
library(glmnet)
library(dplyr)
#library(psych) 
library(lme4)
library(e1071)
library(GGally)
options(warn=-1)
options(scipen=999)
#library(crosstable)

In [ ]:
when N N -> N N
when A N -> N N

when N N -> N N
when A N -> A N


In [7]:
seed_list<-c(1001,1111,1221,1331,1441,1551,1661,1771,1881,1991)
cut_off<-c(0,10,50,100,500,1000)
time_off<-c(10000,10,20,50,100)
#time_off<-c(10)

lambda <- 10^seq(-3, 3, length = 100)
alpha<-seq(0,1,length=10)

In [8]:
library(doParallel)
cl <- makePSOCKcluster(30)
registerDoParallel(cl)

In [9]:
m<-1
v<-1
#list_of_vi <- vector(mode="list", length=5)
#names(list_of_vi) <- c('10000', '10', '20', '50', '100')

list_of_vi_10000<-list()
list_of_vi_10<-list()
list_of_vi_20<-list()
list_of_vi_50<-list()
list_of_vi_100<-list()

corpus_list<-c("google")
list_of_rsqr=list()
ratings_list<-c("reddy","cordeiro90","cordeiro100")
tagged_list<-c("Tagged")
compounds_list<-c("all","an","nn")
ppmi_setting_list<-c("PPMI")
comp_setting_list<-c("Aware","Agnostic")
feature_setting_list<-c("withSetting")
caret_spearman <- function(data, lev = NULL, model = NULL) {
  spearman_val <- cor(x = data$pred, y = data$obs, method = "spearman")
  c(Spearman = spearman_val)
}

In [10]:
for (c in corpus_list){
  for (t in tagged_list) {
    for (f in feature_setting_list) {
      for (p in ppmi_setting_list){
        for (a in comp_setting_list){
          for (i in time_off) {
            
            for (j in cut_off) {
              
              
              print(paste0(c," ",p," ",a," ",f," ",t," ",i," ",j))
              
              na_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_na.csv"),sep = '\t')
              nona_df<-read.csv(paste0("/data/dharp/compounds/datasets/",c,"/features_Compound",a,"_",f,"_",p,"_",t,"_",i,"_",j,"_med.csv"),sep = '\t')
              
              for (r in ratings_list){
                na_df<-na_df %>% filter(source==r)
                nona_df<-nona_df %>% filter(source==r)
                
                for (k in compounds_list) {
                  if (k=="an") { 
                    group1_na<-na_df %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_na<-na_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    na_df<-rbind(group1_na,group2_na)
                    
                    group1_nona<-nona_df %>% filter(is_adj=="True" & grepl("_ADJ",modifier) & grepl("_NOUN",head))
                    group2_nona<-nona_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    nona_df<-rbind(group1_nona,group2_nona)
                    
                  }
                  else if (k=="nn") {
                    group1_na<-na_df %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_na<-na_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    na_df<-rbind(group1_na,group2_na)
                    
                    group1_nona<-nona_df %>% filter(is_adj=="True" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    group2_nona<-nona_df %>% filter(is_adj=="False" & grepl("_NOUN",modifier) & grepl("_NOUN",head))
                    nona_df<-rbind(group1_nona,group2_nona)
                    
                  }
                  
                  trainX_na<-na_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  trainY_na<-na_df %>% select(compositionality)
                  trainY_na<-trainY_na$compositionality
                  
                  
                  trainX_nona<-nona_df %>% select(-c(compositionality,modifier,head,avgModifier,stdevModifier,avgHead,stdevHead,stdevHeadModifier,compound,is_adj,source)) %>% select(-one_of("comp_freq_bins"))
                  trainY_nona<-nona_df %>% select(compositionality)
                  trainY_nona<-trainY_nona$compositionality
                  
                  if (dim(trainX_nona)[1]<10) {
                    break
                  }
                  for (s in seed_list)  {
                    set.seed(s)
                    seeds <- vector(mode = "list", length = 14)
                    for(z in 1:13) seeds[[z]] <- sample.int(n=1000, 10)
                    #for the last model
                    seeds[[14]]<-sample.int(1000, 1)
                    
                    ridge_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                    )
                    
                    elastic_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                          trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                          preProc = c("nzv","center", "scale")
                    )
                    
                    lasso_nona <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                        preProc = c("nzv","center", "scale")
                    )
                    
                    
                    ridge_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv",number = 10, search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    elastic_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                        trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                        preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    lasso_na <- train(trainX_na,trainY_na,method = "glmnet",metric = "Rsquared",
                                      trControl = trainControl("cv", number = 10,search="grid",seeds=seeds),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                      preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    
                    ridge_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                    )
                    
                    elastic_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                   trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                   preProc = c("nzv","center", "scale")
                    )
                    
                    lasso_nona_spearman <- train(trainX_nona,trainY_nona,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                                 preProc = c("nzv","center", "scale")
                    )
                    
                    
                    ridge_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv",number = 10, search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 0, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    elastic_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                                 trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = alpha, lambda = lambda),
                                                 preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    lasso_na_spearman <- train(trainX_na,trainY_na,method = "glmnet",metric = "Spearman",
                                               trControl = trainControl("cv", number = 10,search="grid",seeds=seeds,summaryFunction = caret_spearman),tuneGrid = expand.grid(alpha = 1, lambda = lambda),
                                               preProcess = c("nzv","medianImpute", "center", "scale")
                    )
                    
                    
                    perf_elastic_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_na)[,"method"],TrainRsquared=getTrainPerf(elastic_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_na_spearman)[,"TrainSpearman"])
                    perf_ridge_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_na)[,"method"],TrainRsquared=getTrainPerf(ridge_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_na_spearman)[,"TrainSpearman"])
                    perf_lasso_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_na)[,"method"],TrainRsquared=getTrainPerf(lasso_na)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_na_spearman)[,"TrainSpearman"])
                    
                    
                    perf_elastic_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(elastic_nona)[,"method"],TrainRsquared=getTrainPerf(elastic_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(elastic_nona_spearman)[,"TrainSpearman"])
                    perf_ridge_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(ridge_nona)[,"method"],TrainRsquared=getTrainPerf(ridge_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(ridge_nona_spearman)[,"TrainSpearman"])
                    perf_lasso_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,method=getTrainPerf(lasso_nona)[,"method"],TrainRsquared=getTrainPerf(lasso_nona)[,"TrainRsquared"],TrainSpearman=getTrainPerf(lasso_nona_spearman)[,"TrainSpearman"])
                    
                    varimp_elastic_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_na)$importance))
                    varimp_ridge_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_na)$importance))
                    varimp_lasso_na<-data.frame(feature=f,impute="r",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_na),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_na)$importance))
                    
                    varimp_elastic_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="elastic",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(elastic_nona)$importance))
                    varimp_ridge_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="rigde",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(ridge_nona)$importance))
                    varimp_lasso_nona<-data.frame(feature=f,impute="python",tag=k,setting=a,dataset=r,corpus=c,ml_algo="lasso",n=nrow(trainX_nona),seed=s,timespan=i,cutoff=j,ppmi=p,t(varImp(lasso_nona)$importance))    
                    
                    
                    
                    list_of_rsqr[[m]]<-perf_elastic_na
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_elastic_nona
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_ridge_na
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_ridge_nona
                    m<-m+1
                    
                    list_of_rsqr[[m]]<-perf_lasso_na
                    m<-m+1 
                    
                    list_of_rsqr[[m]]<-perf_lasso_nona
                    m<-m+1 
                    
                    if (i==10000) {
                      list_of_vi_10000[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_10000[[v]]<-varimp_lasso_nona
                      v<-v+1
                      
                    }
                    else if (i==10) {
                      list_of_vi_10[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_10[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    
                    else if (i==20) {
                      list_of_vi_20[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_20[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    else if (i==50) {
                      list_of_vi_50[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_50[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                    else if (i==100) {
                      list_of_vi_100[[v]]<-varimp_elastic_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_elastic_nona
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_ridge_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_ridge_nona
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_lasso_na
                      v<-v+1
                      list_of_vi_100[[v]]<-varimp_lasso_nona
                      v<-v+1
                    }
                  }
                }
                
              }
            }
          }
        }
      }
    }
  }
}

[1] "google PPMI Aware withSetting Tagged 10000 0"
[1] "google PPMI Aware withSetting Tagged 10000 10"
[1] "google PPMI Aware withSetting Tagged 10000 50"
[1] "google PPMI Aware withSetting Tagged 10000 100"
[1] "google PPMI Aware withSetting Tagged 10000 500"
[1] "google PPMI Aware withSetting Tagged 10000 1000"
[1] "google PPMI Aware withSetting Tagged 10 0"
[1] "google PPMI Aware withSetting Tagged 10 10"
[1] "google PPMI Aware withSetting Tagged 10 50"
[1] "google PPMI Aware withSetting Tagged 10 100"
[1] "google PPMI Aware withSetting Tagged 10 500"
[1] "google PPMI Aware withSetting Tagged 10 1000"


In [11]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
statistics<-rsquared_df %>% group_by(impute,setting,dataset,ml_algo,timespan,cutoff) %>% 
summarise(n=mean(n),meanRsquared=mean(TrainRsquared),sdRsquared=sd(TrainRsquared),meanCorr=mean(TrainSpearman),sdCorr=sd(TrainSpearman))

statistics %>% arrange(desc(meanCorr)) %>% head()

`summarise()` has grouped output by 'impute', 'setting', 'dataset', 'ml_algo', 'timespan'. You can override using the `.groups` argument.


impute,setting,dataset,ml_algo,timespan,cutoff,n,meanRsquared,sdRsquared,meanCorr,sdCorr
<chr>,<chr>,<chr>,<chr>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
r,Aware,reddy,elastic,10000,100,182,0.4866613,0.08740117,0.6786246,0.06845876
r,Aware,reddy,elastic,10000,50,182,0.4817285,0.07758526,0.6748800,0.06155756
python,Aware,reddy,elastic,10000,100,182,0.4851510,0.08370846,0.6730310,0.06783599
r,Aware,reddy,elastic,10000,10,182,0.4796296,0.07640544,0.6714369,0.05957625
r,Aware,reddy,elastic,10000,0,182,0.4796292,0.07640610,0.6714318,0.05958310
python,Aware,reddy,elastic,10000,0,182,0.4800318,0.07458213,0.6700133,0.05964572


In [33]:
varimp_df<-bind_rows(list_of_vi)
varimp_df$cutoff<-as.factor(varimp_df$cutoff)
varimp_df[is.na(varimp_df)] <- 0

In [38]:
list_of_vi[1]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,⋯,sim_with_modifier.1920,sim_with_modifier.1930,sim_with_modifier.1940,sim_with_modifier.1950,sim_with_modifier.1960,sim_with_modifier.1970,sim_with_modifier.1980,sim_with_modifier.1990,sim_with_modifier.2000,sim_with_modifier.2010
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Overall,withSetting,r,Tagged,Aware,reddy,google,elastic,364,1001,10,⋯,69.33988,0,0,0,0,0,0,0,0,10.79361


In [7]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)

In [9]:
rsquared_df<-bind_rows(list_of_rsqr)
rsquared_df$cutoff<-as.factor(rsquared_df$cutoff)
rsquared_df[order(-rsquared_df$TrainSpearman), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
644,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1771,20,50,PPMI,glmnet,0.5279124,0.7313636
476,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,100,PPMI,glmnet,0.5225827,0.7200233
356,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1991,10,10,PPMI,glmnet,0.5231564,0.7131652
175,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,50,PPMI,glmnet,0.5585807,0.7128805
177,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,50,PPMI,glmnet,0.5399065,0.7128805
422,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1001,10,100,PPMI,glmnet,0.5311879,0.7118798
235,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,100,PPMI,glmnet,0.5597754,0.7109973
55,woSetting,r,UnTagged,Aware,reddy,google,elastic,91,1991,10000,0,PPMI,glmnet,0.5582943,0.7095471
57,woSetting,r,UnTagged,Aware,reddy,google,rigde,91,1991,10000,0,PPMI,glmnet,0.5399696,0.7095471


In [10]:
rsquared_df[order(-rsquared_df$TrainRsquared), ]

,feature,impute,tag,setting,dataset,corpus,ml_algo,n,seed,timespan,cutoff,ppmi,method,TrainRsquared,TrainSpearman
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<fct>,<chr>,<chr>,<dbl>,<dbl>
440,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,100,PPMI,glmnet,0.6028381,0.6806244
620,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,50,PPMI,glmnet,0.5885712,0.6776766
320,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,10,PPMI,glmnet,0.5854357,0.6588983
380,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,10,50,PPMI,glmnet,0.5837606,0.6813442
324,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,10,PPMI,glmnet,0.5819766,0.6440176
444,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,100,PPMI,glmnet,0.5814872,0.6782434
624,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,20,50,PPMI,glmnet,0.5751432,0.6693253
384,woSetting,python,UnTagged,Aware,reddy,google,lasso,91,1331,10,50,PPMI,glmnet,0.5744349,0.6491037
680,woSetting,python,UnTagged,Aware,reddy,google,elastic,91,1331,20,100,PPMI,glmnet,0.5685864,0.6998218


In [19]:
write.csv(rsquared_df,"rsquared_Tagged_RAW_Agnostic.csv",row.names = FALSE)

In [20]:
write.csv(varimp_df,"varimp_Tagged_RAW_Agnostic.csv",row.names = FALSE)